In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# import jtplot submodule from jupyterthemes
from jupyterthemes import jtplot
# currently installed theme will be used to
# set plot style if no arguments provided
jtplot.style()

from preprocessing import *

data_path = Path('tbmData/data')
fn_txt = sorted(data_path.glob('*.txt'))
print(fn_txt[:3])
fn_cycles = Path('tbmData/cycles1')

mulr = 50
num_cycles = 3481
# num_cycles = 1000
valid_ratio = 0.2
train_ratio = 1 - valid_ratio
train_idx = np.arange(int(num_cycles * valid_ratio), num_cycles)
valid_idx = np.arange(int(num_cycles * valid_ratio))
train_idx_tile = (train_idx[:, None] + np.arange(mulr) * num_cycles).flatten()
valid_idx_tile = (valid_idx[:, None] + np.arange(mulr) * num_cycles
                  ).flatten()  # take from all tiles

bs = int(num_cycles * train_ratio)
gpu_start = 1
num_gpus = 2
torch.cuda.set_device(gpu_start)
device_ids = range(gpu_start, gpu_start + num_gpus)

# cont_names = ['推进速度', '主驱动1#电机扭矩', '刀盘扭矩', '刀盘转速','主液压油箱温度', '前点偏差X', '主液压油箱温度']
dep_var = '推进速度电位器设定值'

In [ ]:
# with concurrent.futures.ThreadPoolExecutor() as e:
#     cycles = tqdm_notebook(e.map(read_feather_fn, sorted(fn_cycles.glob('cycle*'))[:num_cycles]), desc='read_dataframe', total=num_cycles)
#     idx = list(tqdm_notebook(e.map(beginning_index, [o.iloc[:500] for o in cycles]), desc='beginning', total=num_cycles))
# #     cont = [o.loc[:,cont_names].iloc[i:i+30] for i, o in zip(idx, cycles)]
#     cont = [o.iloc[:,3:].iloc[i:i+30] for i, o in zip(idx, cycles)]

In [ ]:
fns_feathers = sorted(fn_cycles.glob('cycle*'))[:num_cycles]
fmtr = DataFormatter(cycle_feathers=fns_feathers)
cycles = fmtr.cycles
idx = [beginning_index(o.iloc[:500]) for o in tqdm_notebook(cycles)]

In [ ]:
cont_names = [o for o in cycles[0].columns[2:] if o != dep_var]
df_conts = tile_with_noise(cycles, idx, mulr)

In [ ]:
# cyc = cycles[0].iloc[:500]
# axis = plots(get_interesting_columns(cyc), title=str(idx));
# for ax in axis.flatten():
#     ax.axvline(idx)

In [ ]:
# deps = pd.DataFrame({'桩号': [o['桩号'].iloc[0] for o in cycles]})
deps = fmtr.get_y(is_problem1=True).iloc[:,0]
deps = pd.Series(np.tile(deps.values, mulr), name=deps.name)
cyc_cont = flatten_and_cat(df_conts, deps)

In [ ]:
cyc_cont.to_feather('tmp/cyc_cont_all_allc')

In [ ]:
cyc_cont = feather.read_dataframe('tmp/cyc_cont_all_5c')

In [ ]:
train, val = cyc_cont.iloc[train_idx_tile], cyc_cont.iloc[valid_idx_tile]
train_x, train_y = train.drop(columns=dep_var), train.loc[:,dep_var]
val_x, val_y = val.drop(columns=dep_var), val.loc[:,dep_var]

In [ ]:
from sklearn.ensemble import forest
from sklearn.ensemble import RandomForestRegressor
def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))

def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))
    
X_train, y_train = train_x, train_y
X_valid, y_valid = val_x, val_y
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
# set_rf_samples(000)
reset_rf_samples()

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, oob_score=False, min_samples_leaf=3, max_features=0.5)
m.fit(train_x, train_y)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=3)
m.fit(train_x, train_y)
print(m.score(train_x, train_y), m.score(val_x, val_y), m.oob_score_)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, oob_score=True, min_samples_leaf=3, max_features=0.5)
m.fit(train_x, train_y)
print(m.score(train_x, train_y), m.score(val_x, val_y), m.oob_score_)

In [ ]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

In [ ]:
list(zip(m.predict(val_x).tolist(), val_y))